In [1]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster import evaluate
from krx_fr.cluster.optimize import Optimizer
from krx_fr.cluster.kmeans import MyKmeans

In [2]:
model = "5호모델의모델"

# 윈도우는 Y로 고정
window_size = "Y"

# 어떻게 윈도우를 밀건지
freq = "Y"

# 클러스터 수
num_k = 3

# 사용할 스케일러
sca = scaler.new_rank
sc = "new_rank"

kospi_listed = tools.load_json("..\\final_data\\코스피_구성종목.json")
kosdaq_listed = tools.load_json("..\\final_data\\코스닥_구성종목.json")
together_listed = tools.aggregate_listed_code(kospi_listed, kosdaq_listed)

# 모델명
model_name = "{}_{}_k{}".format(model, freq, num_k)
model_name

'5호모델의모델_Y_k3'

In [3]:
# 퐅더에서 row_data 불러오기
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\재무\\")

# 최대주주비율은 1달 단위의 pct_change
financial_data2 = tools.dict_data_calculate_pct(financial_data.copy(), name="최대주주비율", return_original=True)

# label_data 불러오기
label_data_war = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\투자주의환기종목지정카운트.csv", index_col=0)
label_data_man= pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\관리종목카운트.csv", index_col=0)
label_data_emb = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\횡령배임혐의발생카운트.csv", index_col=0)

In [4]:
financial_data2["결산기간_주가변동률"] = financial_data2["결산기간_주가변동률"].fillna(0)

In [5]:
# 레이블 데이터 shift시켜주기
label_data_man_shift = tools.shift_label_data(label_data_man.copy(),shift_size='Y')
label_data_war_shift = tools.shift_label_data(label_data_war.copy(),shift_size='Y')
label_data_emb_shift = tools.shift_label_data(label_data_emb.copy(),shift_size='Y')

label_data_pp1 = tools.dict_data_preprocess_upgrade(label_data_man_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_man = tools.dict_data_scale(label_data_pp1.copy(), scaler.zero_one)

label_data_pp2 = tools.dict_data_preprocess_upgrade(label_data_war_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_war = tools.dict_data_scale(label_data_pp2.copy(), scaler.zero_one)

label_data_pp3 = tools.dict_data_preprocess_upgrade(label_data_emb_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_emb = tools.dict_data_scale(label_data_pp3.copy(), scaler.zero_one)

In [6]:
# 데이터처리 sum / mean
kind_data2 = tools.dict_data_preprocess_upgrade(kind_data.copy(), function=np.sum, window_size=window_size,rolling_size=freq, fillna=True)
financial_data3 = tools.dict_data_preprocess_upgrade(financial_data2.copy(), function=np.mean, window_size=window_size, rolling_size=freq,fillna=False)

In [7]:
# 재무데이터와 공시데이터 합치기
concat_data = tools.dict_data_merge(kind_data2.copy(),financial_data3.copy(), dropna=False)

In [8]:
list_data = tools.dict_data_match_listed_upgrade(concat_data.copy(), together_listed.copy(), rolling_size=freq)

In [9]:
# 컬럼을 더할 경우
concat_together1 = tools.dict_data_plus(list_data.copy(),
                                  plus_one="BW금액"
                                 ,plus_two="CB금액", return_origianl=False)

#
concat_together1["2012-01"].columns

Index(['BW금액+CB금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '유상증자금액',
       '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [10]:
concat_together2 = tools.dict_data_plus(concat_together1, 
                                  plus_one="BW금액+CB금액"
                                 ,plus_two="유상증자금액", return_origianl=False)
#concat_kospi2 = tools.dict_data_plus(concat_kospi1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq2 = tools.dict_data_plus(concat_kosdaq1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
concat_together2["2012-01"].columns

Index(['BW금액+CB금액+유상증자금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트',
       '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [11]:
concat_together3 = tools.dict_data_plus(concat_together2, 
                                  plus_one="최대주주변경을수반하는주식담보제공금액"
                                 ,plus_two="최대주주변경을수반하는주식양수도금액", return_origianl=False)
#concat_kospi3 = tools.dict_data_plus(concat_kospi2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kosdaq3 = tools.dict_data_plus(concat_kosdaq2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#
concat_together3["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '조회공시요구카운트', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [12]:
concat_together4 = tools.dict_data_plus(concat_together3, 
                                  plus_one="타법인주식및출자증권취득결정금액"
                                 ,plus_two="특수관계인에대한자금대여금액", return_origianl=False)
#concat_kospi4 = tools.dict_data_plus(concat_kospi3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
#concat_kosdaq4 = tools.dict_data_plus(concat_kosdaq3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
concat_together4["2012-01"].columns

Index(['타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '조회공시요구카운트', '최대주주변경카운트',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [13]:
concat_together5 = tools.dict_data_plus(concat_together4,
                                  plus_one="상호변경카운트"
                                 ,plus_two="업종변경카운트", return_origianl=False)
#concat_kospi5 = tools.dict_data_plus(concat_kospi4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
#concat_kosdaq5 = tools.dict_data_plus(concat_kosdaq4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
concat_together5["2012-01"].columns

Index(['상호변경카운트+업종변경카운트', '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '조회공시요구카운트', '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [14]:
concat_together6 = tools.dict_data_plus(concat_together5,
                                  plus_one="경영권분쟁소송"
                                 ,plus_two="최대주주변경카운트", return_origianl=False)
#concat_kospi6 = tools.dict_data_plus(concat_kospi5,
#                                  plus_one="경영권분쟁소송_전체시장"
#                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
#concat_kosdaq6 = tools.dict_data_plus(concat_kosdaq5,
#                                  plus_one="경영권분쟁소송_전체시장"
#                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
concat_together6["2012-01"].columns

Index(['경영권분쟁소송+최대주주변경카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '불성실공시법인지정카운트', '조회공시요구카운트', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익',
       '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [15]:
concat_together7 = tools.dict_data_plus(concat_together6,
                                  plus_one="경영권분쟁소송+최대주주변경카운트"
                                 ,plus_two="불성실공시법인지정카운트", return_origianl=False)
#concat_kospi7 = tools.dict_data_plus(concat_kospi6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
#concat_kosdaq7 = tools.dict_data_plus(concat_kosdaq6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
concat_together7["2012-01"].columns

Index(['경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '조회공시요구카운트', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [16]:
concat_together8 = tools.dict_data_plus(concat_together7,
                                  plus_one="경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트"
                                 ,plus_two="조회공시요구카운트", return_origianl=False)
#concat_kospi8 = tools.dict_data_plus(concat_kospi7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq8 = tools.dict_data_plus(concat_kosdaq7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
concat_together8["2012-01"].columns

Index(['경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [17]:
concat_together9 = tools.dict_data_plus(concat_together8,
                                  plus_one="경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트"
                                 ,plus_two="상호변경카운트+업종변경카운트", return_origianl=False)
concat_together9["2012-01"].columns

Index(['경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트+상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [18]:
concat_together10 = tools.dict_data_plus(concat_together9,
                                  plus_one="최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액"
                                 ,plus_two="타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액", return_origianl=False)
concat_together10["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액+타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트+상호변경카운트+업종변경카운트',
       'BW금액+CB금액+유상증자금액', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권',
       '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비',
       '최대주주비율_변동성'],
      dtype='object')

In [19]:
concat_together11 = tools.dict_data_plus(concat_together10,
                                  plus_one="최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액+타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액"
                                 ,plus_two="BW금액+CB금액+유상증자금액", return_origianl=False)
print(concat_together11["2012-01"].columns.shape)
concat_together11["2012-01"].columns

(14,)


Index(['최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액+타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액+BW금액+CB금액+유상증자금액',
       '경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트+상호변경카운트+업종변경카운트',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [20]:
drop_togehter = tools.dict_data_drop(concat_together11.copy(), dropna=True)
scaled_together = tools.dict_data_scale(drop_togehter.copy(), scaler=sca)
scaled_together["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액+타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액+BW금액+CB금액+유상증자금액',
       '경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트+상호변경카운트+업종변경카운트',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [21]:
# 최초 모델을 돌리고
model_save_dict = evaluate.run_iter_kmeans(num_k, max_sample=100, raw_data=scaled_together.copy())

100%|██████████| 100/100 [00:07<00:00, 13.46it/s]


In [22]:
# 최초모델의 결과
eval_one, eval_one_original = evaluate.eval_cluster_result(model_save_dict,
                                                            raw_data = scaled_together, 
                                                            label_data=label_data_pp_man, freq=freq)

# 모델의 모델(나쁜놈들도 다시한번 굴림)
model2_save_dict, filtered_data = evaluate.cluster_of_cluster_model(model_save=model_save_dict,
                                  raw_data=scaled_together,
                                  eval_label=eval_one, num_k=3, max_sample=100)

100%|██████████| 100/100 [00:01<00:00, 60.70it/s]


### 모델의 모델 평가

In [23]:
size =evaluate.eval_cluster_size(model2_save_dict)
size.to_csv("{}_클러스터사이즈.csv".format(model_name))

print(size.iloc[:-1,:].max().max())
print(size.iloc[:-1,:].min().min())
size

255.0
124.0


,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,203.0,184.0,144.0,188.0,173.0,207.0,233.0,200.0,238.0,219.0
1,153.0,189.0,176.0,152.0,219.0,192.0,197.0,198.0,199.0,255.0
2,143.0,124.0,177.0,193.0,181.0,187.0,177.0,173.0,249.0,241.0
기업수,499.0,497.0,497.0,533.0,573.0,586.0,607.0,571.0,686.0,715.0


In [24]:
eval, eval_original = evaluate.eval_cluster_result(model2_save_dict, raw_data = filtered_data,
                                                    label_data=label_data_pp_man, freq=freq)

eval.to_csv("{}_관리종목.csv".format(model_name))
eval_original.to_csv("{}_관리종목전체대비.csv".format(model_name))

In [25]:
eval_t,eval_t_ori = evaluate.eval_cluster_result(model2_save_dict, raw_data = filtered_data, 
                                                 label_data=label_data_pp_war, freq=freq)

eval_t.to_csv("{}_투자환기종목.csv".format(model_name))
eval_t_ori.to_csv("{}_투자환기종목전체대비.csv".format(model_name))

In [26]:
print(evaluate.calculate_catch_probability(eval, eval_t, None))
print(evaluate.calculate_match_probability(eval, eval_t,None))
evaluate.calculate_catch_probability(eval_original, eval_t_ori, None)

0.6045345436926283
1.0


0.06625672923032155

In [27]:
def pct(df):
    return df / np.sum(df)

dist = evaluate.eval_cluster_distance(eval, filtered_data, model2_save_dict)
dist = dist.apply(pct, axis=1)
dist2= dist.mean(axis=0).sort_values(ascending=False)

dist.to_csv("{}_거리분해.csv".format(model_name))
dist2.to_csv("{}_거리분해평균.csv".format(model_name))

dist2

경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트+상호변경카운트+업종변경카운트                                   0.216771
결산기간_거래량증가                                                                                 0.155465
차입금의존도                                                                                     0.143905
최대주주비율_변동성                                                                                 0.119584
최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액+타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액+BW금액+CB금액+유상증자금액    0.103679
최대주주비율                                                                                     0.068131
판매 및 관리비                                                                                   0.049613
매출채권                                                                                       0.042147
당기순이익                                                                                      0.030563
결산기간_주가변동률                                                                                 0.026857


고위험 클러스터의 평균 재무, 공시 상황 출력

* eval_cluster Result의 결과를 받는다 / model_save_dict / raw_data -> drop_together (스케일 전 데이터를 받아야함)

In [28]:
# scaled data를 토대로 원본 데이터를 추출한다
key = filtered_data.keys()
original_data_dict = {}
for t in key:
    idx = filtered_data[t].index
    origianl_data = drop_togehter[t]
    original_data_dict[t] = origianl_data.loc[idx]

In [29]:
dict = evaluate.see_cluster_info(model2_save_dict, eval, original_data_dict)
data = pd.DataFrame()
col = dict.keys()

for t, df in dict.items():
    data = pd.concat([data, df], axis=1)

data.columns = col
data.T.to_csv("{}_평균정보.csv".format(model_name))
data.T

,최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액+타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액+BW금액+CB금액+유상증자금액,경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트+상호변경카운트+업종변경카운트,결산기간_거래량증가,결산기간_주가변동률,당기순이익,매출채권,매출총이익,영업이익,영업현금흐름,차입금의존도,총자본,최대주주비율,판매 및 관리비,최대주주비율_변동성
2012-01,1.654002e+10,1.196078,1.329681,1.951553,-6.746307e+06,3.454802e+07,-5.762215e+05,-4.787962e+06,-7.890767e+05,43.281193,6.826823e+07,26.069136,4.211740e+06,-0.014554
2013-01,2.031735e+10,1.411290,1.221427,4.390568,-2.434947e+07,1.092742e+08,-6.129393e+06,-1.433816e+07,-5.738663e+06,45.619738,1.029610e+08,28.927098,8.210083e+06,-0.004517
2014-01,2.274832e+10,2.368056,2.068829,3.596152,-1.779983e+07,3.020803e+07,-5.223987e+06,-8.013657e+06,-3.609888e+06,30.829913,5.919321e+06,24.250851,2.789670e+06,inf
2015-01,2.060649e+10,2.138158,1.151989,2.068483,-6.244506e+06,3.344949e+07,-1.399430e+06,-4.321868e+06,-1.639472e+06,32.684688,3.657043e+07,25.094616,2.922438e+06,inf
2016-01,1.423145e+10,1.922652,inf,0.793543,-1.305338e+06,1.131377e+07,1.766077e+06,-6.844295e+05,-2.477429e+05,25.928854,3.557186e+07,28.539759,2.450506e+06,-0.023327
2017-01,3.223311e+10,2.416667,inf,1.186597,-1.955092e+07,2.096177e+07,5.617642e+05,-3.366167e+06,2.702154e+04,31.743047,3.404551e+07,18.081104,3.927931e+06,inf
2018-01,4.424966e+10,1.858369,inf,1.071338,-4.213075e+06,1.873886e+07,1.696640e+06,-2.220785e+06,-1.779292e+06,34.030923,5.552423e+07,20.918990,3.917425e+06,0.005981
2019-01,3.439317e+10,2.575000,1.735226,0.515285,-3.909257e+06,1.156527e+07,2.066872e+06,-1.342674e+06,-7.748843e+05,34.229988,4.546267e+07,19.565421,3.409539e+06,inf
2020-01,3.357250e+10,1.610442,inf,0.624480,-9.330883e+06,1.743279e+07,5.634536e+05,-4.956890e+06,-2.330407e+06,41.334488,7.873014e+07,20.574489,5.516598e+06,inf
2021-01,2.914067e+10,1.867580,inf,0.499985,-2.426894e+06,6.912510e+06,1.267869e+06,-1.136530e+06,-7.483508e+05,25.808961,4.651702e+07,20.628501,2.408825e+06,inf


In [30]:
data.T.mean().to_csv("{}_평균정보의평균.csv".format(model_name))

In [31]:
# 그림뽑기

test = MyKmeans.visualize_3d_with_answer(filtered_data["2020-01"],
                                        model2_save_dict["2020-01"]["model"],
                                         answer_data=label_data_pp_man["2020-01"],
                                         fig_title="{}_관리종목".format(model_name))

In [32]:
MyKmeans.visualize_3d_with_answer(filtered_data["2020-01"],
                                  model2_save_dict["2020-01"]["model"],
                                 answer_data=label_data_pp_war["2020-01"],
                                 fig_title="{}_투자주의환기".format(model_name))

,PC1,PC2,PC3,label,answer
A000040,0.315993,0.093533,-0.324416,2,2.5
A000440,-0.570949,0.135473,-0.254692,0,2.5
A000650,-0.462358,-0.139028,0.119666,1,2.5
A000890,0.288320,0.241414,-0.259987,2,2.5
A000950,0.456446,-0.309818,0.151462,2,2.5
...,...,...,...,...,...
A317850,-0.002634,-0.369613,-0.195786,1,2.5
A317870,-0.318887,0.057784,-0.493458,0,2.5
A318000,-0.811293,0.393391,-0.136025,0,2.5
A322510,0.140256,-0.186774,0.096634,1,2.5
